1.6.1: Functions as Arguments

In [24]:
def summation(n, func):
    k, res = 1, 0
    while k <= n:
        res, k = res + func(k), k + 1
    return res

def cube(x):
    return x*x*x

def identity(x):
    return x

def pi_term(x):
    return 8/((4 * x - 3) * (4 * x - 1))

def sum_cubes(n):
    return summation(n, cube)

def sum_naturals(n):
    return summation(n, identity)

def pi_sum(n):
    return summation(n, pi_term)

In [25]:
sum_cubes(10)

3025

In [26]:
sum_naturals(10)

55

In [27]:
pi_sum(1000)

3.141092653621038

1.6.2 Functions as General Methods

In [28]:
def improve(close, update, guess = 1):
    while not close(guess):
        guess = update(guess)
    return guess

In [29]:
def golden_update(guess):
    return 1/guess + 1

def golden_close(guess):
    return approx_eq(guess * guess, guess + 1)

def approx_eq(x, y, threshold = 1e-15):
    return abs(x - y) < threshold

In [30]:
improve(golden_close, golden_update, -100000000)

1.6180339887498951

1.6.3: Defining Functions III: Nested Definitions

In [31]:
def sqrt(a):
    def sqrt_update(x):
        return (x + a/x) / 2
    def sqrt_close(x):
        return approx_eq(x * x, a)
    return improve(sqrt_close, sqrt_update)

In [32]:
sqrt(9)

3.0

1.6.4: Functions as Returned Values

In [33]:
""" def compose1(f, g):
    def h(x):
        return f(g(x))
    return h """

In [34]:
def square(x):
    return x * x

def successor(x):
    return x + 1

In [35]:
successor_squared = compose1(square, successor)
successor_squared(12)

169

In [36]:
def make_adder(f):
    def adder(k):
        return f + k
    return adder

In [37]:
add3 = make_adder(3)
add3(2) 

5

1.6.5: Example Newton's Method

In [38]:
def newton_update(f, df):
    def update(x):
        return x - f(x)/df(x)
    return update

def find_zero(f, df):
    def near_zero(x):
        return approx_eq(f(x), 0)
    return improve(near_zero, newton_update(f, df))

In [39]:
def square_root_newton(a):
    def f(x):
        return x * x - a
    def df(x):
        return 2 * x
    return find_zero(f, df)

In [40]:
square_root_newton(64)

8.0

In [41]:
def power(x, n):
    product, k = 1, 0
    while k < n:
        product, k = product * x, k + 1
    return product

In [43]:
def nth_root_newton(a, n):
    def f(x):
        return power(x, n) - a
    def df(x):
        return n * power(x, n - 1)
    return find_zero(f, df)

In [44]:
nth_root_newton(100, 2)

10.0

1.6.6: Currying

In [45]:
def curried_pow(x):
    def h(y):
        return pow(x, y)
    return h


In [50]:
curried_pow(5)(5)

3125

In [51]:
def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start += 1

In [52]:
map_to_range(0, 10, curried_pow(5))

1
5
25
125
625
3125
15625
78125
390625
1953125


In [53]:
def curry2(f):
    """Return a curried version of the given two-argument function."""
    def g(x):
        def h(y):
            return f(x, y)
        return h
    return g

def uncurry2(g):
    """Return a two-argument version of the given curried function."""
    def f(x, y):
        return g(x)(y) 
    return f

In [54]:
pow_curried = curry2(pow)
pow_curried(5)(6)

15625

In [55]:
uncurry2(pow_curried)(5, 6)

15625

1.6.7: Lambda Expressions

In [57]:
def compose1(f, g):
    return lambda x: f(g(x))

"""compose1 = lambda f,g: lambda x: f(g(x))"""

In [60]:
s = lambda x: x * x
s(12)

144

1.6.9: Function Decorators

In [61]:
def trace(fn):
        def wrapped(x):
            print('-> ', fn, '(', x, ')')
            return fn(x)
        return wrapped

In [65]:
h = trace(pow_curried)
h(50)(2)

->  <function curry2.<locals>.g at 0x00000204165A2700> ( 50 )


2500

In [70]:
@trace
def triple(x):
    """The @ decorator affects the def statement, triple is not bound to this function.
    Insted the name triple is bound to the returned function value of calling trace
    on the newly defined triple function. It is equivalent to 
    def triple(x):
        return x * 3
    triple = trace(triple)"""
    return 3 * x

In [69]:
triple(5)

->  <function triple at 0x00000204172AE700> ( 5 )


15